In [ ]:
#--packages org.locationtech.rasterframes:pyrasterframes_2.11:0.7.1,org.locationtech.rasterframes:rasterframes_2.11:0.7.1,org.locationtech.rasterframes:rasterframes-datasource_2.11:0.7.1
import sys
sys.path.append('/home/student/ROI/SparkProgram')
from initspark import *
sc, spark, conf = initspark()

import pandas as pd
import matplotlib as mp
import numpy
from matplotlib import pyplot as plt
from IPython.display import display
import pyspark_ML_helpers as hlp

In [ ]:
#filename = 'CarLoanDefaults.csv'
filename = 'bank.csv'

df = spark.read.csv(f'/home/student/ROI/Spark/datasets/finance/{filename}', header = True, inferSchema = True)
display(df.limit(10).toPandas())

# Save a pointer to the raw data
df0 = df



In [ ]:
import pyspark.sql.functions as F

# Keep all columns in either df1 or df2
def outter_union(df1, df2):

    # Add missing columns to df1
    left_df = df1
    for column in set(df2.columns) - set(df1.columns):
        left_df = left_df.withColumn(column, F.lit(None))

    # Add missing columns to df2
    right_df = df2
    for column in set(df1.columns) - set(df2.columns):
        right_df = right_df.withColumn(column, F.lit(None))

    # Make sure columns are ordered the same
    return left_df.union(right_df.select(left_df.columns))

In [6]:
import imp
imp.reload(hlp)


# from pyspark.ml import Pipeline
# from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler
# from pyspark.sql.functions import monotonically_increasing_id

#df90 = df.select('marital','age', 'deposit').withColumn('id', monotonically_increasing_id())
# df90.show()
# maritalIndexer = StringIndexer(inputCol = 'marital', outputCol = 'maritalIndex')
# maritalIndexerFit = maritalIndexer.fit(df.select('marital'))
# print(maritalIndexerFit.labels)

# encoder = OneHotEncoderEstimator(inputCols=['maritalIndex'], outputCols=['maritalVec'])
# pipeline = Pipeline(stages=[maritalIndexer, encoder])
# df91 = pipeline.fit(df90.select('marital')).transform(df90.select('marital')).withColumn('id', monotonically_increasing_id())
# df91.show()
# df91.printSchema()

# df93 = df90.join(df91, on = 'id').select('maritalVec', 'age', 'deposit') #.drop('id')
# df93.show()

import pyspark_ML_helpers as hlp
from pyspark_ML_helpers import StringIndexEncode, OneHotEncode, AssembleFeatures

# def StringIndexEncode(df, columns):
#     df1 = df
#     for col in columns:
#         indexer = StringIndexer(inputCol = col, outputCol = col+'_Index')
#         df1 = indexer.fit(df1).transform(df1).drop(col) #.withColumn('id', monotonically_increasing_id())
#     return df1


# def OneHotEncode(df, columns):
#     df1 = df
#     for col in columns:
#         encoder = OneHotEncoderEstimator(inputCols=[col + '_Index'], outputCols=[col+'_Vector'])
#         df1 = encoder.fit(df1).transform(df1).drop(col + '_Index')
#     return df1

# def AssembleFeatures(df, categorical_features, numeric_features, label):
#     assemblerInputs = [c + "_Vector" for c in categorical_features] + numeric_features
#     assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
#     return assembler.transform(df).withColumnRenamed(label, 'label').drop(*(numeric_features + [c + '_Vector' for c in categorical_features]))

print ('***********')
df10 = StringIndexEncode(df.select('marital', 'job', 'age', 'deposit'), ['marital','job','deposit'])
df10.show()
df11 = OneHotEncode(df10, ['marital', 'job'])
df11.show()

df12 = AssembleFeatures(df11, ['marital','job'],['age'], 'deposit_Index')
df12.printSchema()

# def collect_tuple(df):
#     return [tuple(row) if len(tuple(row)) > 1 else tuple(row)[0] for row in df.collect()]

# def collect_dict(df):
#     return dict(collect_tuple(df))

# print (collect_tuple(df91.select('marital', 'maritalIndex').distinct()))
# print (collect_dict(df91.select('marital', 'maritalIndex').distinct()))



***********
+---+-------------+---------+-------------+
|age|marital_Index|job_Index|deposit_Index|
+---+-------------+---------+-------------+
| 59|          0.0|      3.0|          1.0|
| 56|          0.0|      3.0|          1.0|
| 41|          0.0|      2.0|          1.0|
| 55|          0.0|      4.0|          1.0|
| 54|          0.0|      3.0|          1.0|
| 42|          1.0|      0.0|          1.0|
| 56|          0.0|      0.0|          1.0|
| 60|          2.0|      5.0|          1.0|
| 37|          0.0|      2.0|          1.0|
| 28|          1.0|      4.0|          1.0|
| 38|          1.0|      3.0|          1.0|
| 30|          0.0|      1.0|          1.0|
| 29|          0.0|      0.0|          1.0|
| 46|          1.0|      1.0|          1.0|
| 31|          1.0|      2.0|          1.0|
| 35|          2.0|      0.0|          1.0|
| 32|          1.0|      1.0|          1.0|
| 49|          0.0|      4.0|          1.0|
| 41|          0.0|      3.0|          1.0|
| 49|          2.0| 

In [9]:
train, test = df12.randomSplit([.7,.3], seed = 1000)
print (f'Training set row count {train.count()}')
print (f'Testing set row count {test.count()}')
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=10)
lrModel = lr.fit(train)
print('LR Trained')
filename1 = filename.replace('.','_') + '_LR_trainedModel'

lrModel.write().overwrite().save(filename1)
#lrModel.save(filename1)
print('LR Saved')

#lrPredictions, log = hlp.predict_and_evaluate(lrModel, test)

#lrModel2 = LogisticRegression.load(filename1)
#print('LR Loaded')



Training set row count 7760
Testing set row count 3402
LR Trained
LR Saved


In [11]:
lrModel2 = LogisticRegression.load(filename1)

Py4JJavaError: An error occurred while calling o900.load.
: java.lang.NoSuchMethodException: org.apache.spark.ml.classification.LogisticRegressionModel.<init>(java.lang.String)
	at java.lang.Class.getConstructor0(Class.java:3082)
	at java.lang.Class.getConstructor(Class.java:1825)
	at org.apache.spark.ml.util.DefaultParamsReader.load(ReadWrite.scala:496)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnec

In [ ]:
if filename == 'bank.csv':
    drop_cols = ('day','month')
    target_col = 'deposit'
    categorical_features = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'campaign','poutcome']
elif filename == 'CarLoanDefaults.csv':
    drop_cols = ('UNIQUEID','DATE_OF_BIRTH','DISBURSAL_DATE', 'AVERAGE_ACCT_AGE', 'CREDIT_HISTORY_LENGTH')
    target_col = 'LOAN_DEFAULT'
    categorical_features = hlp.auto_categorical_features(df)

df = hlp.drop_columns(df0, drop_cols)
numeric_features = hlp.auto_numeric_features(df, exceptlist = categorical_features)
print (numeric_features)
print ('*' * 80)
hlp.describe_numeric_features(df, numeric_features)
print ('*' * 80)

# save a pointer to the fixed data
df1 = df


In [ ]:
#hlp.scatter_matrix(df, numeric_features)

In [ ]:
df, catindexes = hlp.fix_categorical_data(df, categorical_features, target_col, numeric_features)
# save a pointer to this stage of the dataframe
df2 = df
df.printSchema()

print (catindexes)

In [ ]:
train, test = df.randomSplit([.7,.3], seed = 1000)
print (f'Training set row count {train.count()}')
print (f'Testing set row count {test.count()}')
      

In [ ]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=10)
lrModel = lr.fit(train)
print('LR Trained')
lrModel.save(filename.replace('.','_') + '_LR.trainedModel')

In [ ]:
lrPredictions, log = hlp.predict_and_evaluate(lrModel, test)


In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'label', maxDepth = 3)
dtModel = dt.fit(train)
print('DT Trained')
filename1 = filename.replace('.','_') + '_DT_trainedModel'
#dtModel.stages
#.write.overwrite().save(filename1)
dtModel.save(filename1)
print('DT Saved')
dtModel2 = DecisionTreeClassifier.load(filename1)
print('DT Loaded')



In [ ]:
dtPredictions, log = hlp.predict_and_evaluate(gbtModel, test)

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'label')
rfModel = rf.fit(train)
print('RF Trained')
rfModel.save(filename.replace('.','_') + '_RF.trainedModel')

In [ ]:
rfPredictions, log = hlp.predict_and_evaluate(gbtModel, test)

In [ ]:
from pyspark.ml.classification import GBTClassifier
gbt = GBTClassifier(maxIter=10)
gbtModel = gbt.fit(train)
print ('GBT Trained')
gbtModel.save(filename.replace('.','_') + '_GBT.trainedModel')

In [ ]:
#import imp
#imp.reload(hlp)

gbtPredictions, log = hlp.predict_and_evaluate(gbtModel, test)
